In [1]:
import qiskit_aer.noise as noise

# Define error probabilities
prob_1 = 0.001  # 1-qubit gate error probability
prob_2 = 0.01   # 2-qubit gate error probability

# Create depolarizing errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Initialize a noise model
noise_model = noise.NoiseModel()

# Add errors to the noise model
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u1', 'u2', 'u3']
  Instructions with noise: ['u2', 'u1', 'u3', 'cx']
  All-qubits errors: ['u1', 'u2', 'u3', 'cx']


In [3]:
import simpy

class QuantumDevice:
    def __init__(self, env):
        self.env = env

    def apply_gate(self, gate, qubits):
        # Original gate application
        yield self.env.timeout(1)
        print(f"Applied {gate} on {qubits} at {self.env.now}")

    def apply_gate_with_noise(self, gate, qubits, noise_model):
        # Apply noise model
        yield self.env.timeout(1)
        noisy_result = noise_model(gate, qubits)
        print(f"Applied {noisy_result} with noise at {self.env.now}")

def simple_noise_model(gate, qubits):
    # Example noise model
    noise = {}
    noise ["H"] = {}
    noise["H"][(1,)] = 0.15
    noise["H"][(0,)] = 0.32
    
    return f"{gate} with {noise[gate][tuple(qubits)]}"

# Usage
env = simpy.Environment()
q_device = QuantumDevice(env)
env.process(q_device.apply_gate_with_noise("H", [0], simple_noise_model))
env.process(q_device.apply_gate_with_noise("H", [1], simple_noise_model))
env.run(until=6)

Applied H with 0.32 with noise at 1
Applied H with 0.15 with noise at 1


In [ ]:
import pandas as pd

# Load the uploaded CSV file to inspect its structure
file_path = 'QCloud/calibration/ibm_fez_calibrations_2025-01-13T16_54_24Z.csv'
calibration_data = pd.read_csv(file_path)

# Display the first few rows to understand its format
# calibration_data.head()
# calibration_data.columns.tolist()

In [ ]:
def extract_errors_from_csv_fixed(calibration_data):
    # Remove trailing spaces from column names
    calibration_data.columns = calibration_data.columns.str.strip()
    
    # Extract readout errors: list of readout assignment errors for each qubit
    readout_errors = calibration_data["Readout assignment error"].tolist()

    # Extract single-qubit gate errors (using 'RX error' and 'Pauli-X error' as an example)
    single_qubit_gate_errors = {
        "rx": calibration_data["RX error"].mean(),  # Average RX error
        "x": calibration_data["Pauli-X error"].mean()  # Average Pauli-X error
    }

    # Extract two-qubit gate errors from the 'CZ error' column
    # The CZ errors are listed as key-value pairs (e.g., "0_1:0.0093")
    two_qubit_gate_errors = {}
    for cz_errors in calibration_data["CZ error"]:
        pairs = cz_errors.split(";")
        for pair in pairs:
            gate, error = pair.split(":")
            two_qubit_gate_errors[gate] = float(error)

    return readout_errors, single_qubit_gate_errors, two_qubit_gate_errors

# Call the function with corrected column names
readout_errors, single_qubit_gate_errors, two_qubit_gate_errors = extract_errors_from_csv_fixed(calibration_data)

def estimate_fidelity(circuit, single_qubit_gate_errors, two_qubit_gate_errors, readout_errors):
    """
    Estimate fidelity of a quantum circuit using pre-computed gate and readout errors.
    
    Parameters:
    - circuit: QuantumCircuit object.
    - single_qubit_gate_errors: Dictionary with average single-qubit gate error rates.
    - two_qubit_gate_errors: Dictionary with error rates for two-qubit gates.
    - readout_errors: List of readout assignment errors for each qubit.

    Returns:
    - estimated_fidelity: Estimated fidelity of the circuit.
    """
    num_qubits = circuit['num_qubits']
    depth = circuit['depth']

    # Estimate single-qubit gate fidelity
    avg_single_qubit_error = single_qubit_gate_errors["rx"]  # Assuming RX as representative
    single_qubit_fidelity = (1 - avg_single_qubit_error) ** depth

    # Estimate two-qubit gate fidelity

    
    # We don't have data to two-qubit gates yet
    
    for gate in circuit['gates']:
        if gate[0].name == "cx":  # Assuming CX gates have the largest impact
            qubits = gate[1]
            gate_label = f"{qubits[0]}_{qubits[1]}"
            error_rate = two_qubit_gate_errors.get(gate_label, 0.01)  # Default error if missing
            two_qubit_fidelity *= (1 - error_rate)
    

    # Estimate readout fidelity
    avg_readout_error = sum(readout_errors) / len(readout_errors)
    readout_fidelity = (1 - avg_readout_error) ** num_qubits

    # Combined fidelity
    estimated_fidelity = single_qubit_fidelity * two_qubit_fidelity * readout_fidelity
    return estimated_fidelity

In [ ]:
import csv
import json

json_file_path = 'QCloudSimJobs_MQT_ibm2.json'
jobs = []

with open(json_file_path, 'r') as jsonfile:
    data = json.load(jsonfile)
    jobs = data.get("jobs", [])
    
for job in jobs:
    # print(job)
    ef = estimate_fidelity(job, single_qubit_gate_errors, two_qubit_gate_errors, readout_errors)
    print(ef)
